[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/langchain-retrieval-agent.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/langchain-retrieval-agent.ipynb)

#### [LangChain Handbook](https://pinecone.io/learn/langchain)

# Retrieval Agents

Conversational agents can struggle with data freshness, knowledge about specific domains, or accessing internal documentation. By coupling agents with retrieval augmentation tools we no longer have these problems.

One the other side, using "naive" retrieval augmentation without the use of an agent means we will retrieve contexts with *every* query. Again, this isn't always ideal as not every query requires access to external knowledge.

Merging these methods gives us the best of both worlds. In this notebook we'll learn how to do this.

To begin, we must install the prerequisite libraries that we will be using in this notebook.

In [1]:
!pip install -qU \
    openai==0.27.7 \
    pinecone-client==3.1.0 \
    pinecone-datasets==0.7.0 \
    langchain==0.1.1 \
    langchain-community==0.0.13 \
    tiktoken==0.4.0

## Building the Knowledge Base

We will download a pre-embedded dataset from `pinecone-datasets`. Allowing us to skip the embedding and preprocessing steps, if you'd rather work through those steps you can find the [full notebook here](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/08-langchain-retrieval-agent.ipynb).

In [2]:
from pinecone_datasets import load_dataset

dataset = load_dataset("langchain-python-docs-text-embedding-ada-002")
dataset.head()

,id,values,sparse_values,metadata,blob
0.0,417ede5d-39be-498f-b518-f47ed4e53b90,"[0.005949743557721376, 0.01983247883617878, -0...",None,None,"{'chunk': 0, 'text': '.rst .pdf Welcome to Lan..."
1.0,110f550d-110b-4378-b95e-141397fa21bc,"[0.009401749819517136, 0.02443608082830906, 0....",None,None,"{'chunk': 1, 'text': 'Use Cases# Best practice..."
2.0,d5f00f02-3295-4567-b297-5e3262dc2728,"[-0.005517194513231516, 0.0208403542637825, 0....",None,None,"{'chunk': 2, 'text': 'Gallery: A collection of..."
3.0,0b6fe3c6-1f0e-4608-a950-43231e46b08a,"[-0.006499645300209522, 0.0011573900701478124,...",None,None,"{'chunk': 0, 'text': 'Search Error Please acti..."
4.0,39d5f15f-b973-42c0-8c9b-a2df49b627dc,"[-0.005658374633640051, 0.00817849114537239, 0...",None,None,"{'chunk': 0, 'text': '.md .pdf Dependents Depe..."


In [3]:
len(dataset)

6952

We'll format the dataset ready for upsert and reduce what we use to a subset of the full dataset.

In [4]:
# we drop sparse_values as they are not needed for this example
dataset.documents.drop(['sparse_values', 'metadata'], axis=1, inplace=True)
dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)

dataset.documents

,id,values,metadata
0.0,417ede5d-39be-498f-b518-f47ed4e53b90,"[0.005949743557721376, 0.01983247883617878, -0...","{'chunk': 0, 'text': '.rst .pdf Welcome to Lan..."
1.0,110f550d-110b-4378-b95e-141397fa21bc,"[0.009401749819517136, 0.02443608082830906, 0....","{'chunk': 1, 'text': 'Use Cases# Best practice..."
2.0,d5f00f02-3295-4567-b297-5e3262dc2728,"[-0.005517194513231516, 0.0208403542637825, 0....","{'chunk': 2, 'text': 'Gallery: A collection of..."
3.0,0b6fe3c6-1f0e-4608-a950-43231e46b08a,"[-0.006499645300209522, 0.0011573900701478124,...","{'chunk': 0, 'text': 'Search Error Please acti..."
4.0,39d5f15f-b973-42c0-8c9b-a2df49b627dc,"[-0.005658374633640051, 0.00817849114537239, 0...","{'chunk': 0, 'text': '.md .pdf Dependents Depe..."
...,...,...,...
NaN,8019ac6d-786e-4802-9c31-ed11b87c362b,"[0.003017181996256113, 0.01940435729920864, -0...","{'chunk': 2, 'text': '[{'text': '\n\nEnvironme..."
NaN,05087c76-15e4-4abc-b523-2898ba13918a,"[0.008789504878222942, 0.004951421171426773, -...","{'chunk': 3, 'text': 'of any programming langu..."
NaN,7f0ebe3b-bec3-4910-8fda-0c6b7276da09,"[0.0019375714473426342, 0.004722204990684986, ...","{'chunk': 4, 'text': 'important when it comes ..."
NaN,819c1e3e-5cbe-40f2-aaf4-e2196fcf8b19,"[0.01285293698310852, 0.0072921025566756725, 0...","{'chunk': 5, 'text': 'explore both of these op..."


## Creating an Index

Now the data is ready, we can set up our index to store it.

We begin by initializing our connection to Pinecone. To do this we need a [free API key](https://app.pinecone.io).

In [5]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = ""

# configure client
pc = Pinecone(api_key=api_key)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [6]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [7]:
index_name = 'langchain-retrieval-agent-fast'

In [8]:
import time

if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

# we create a new index
pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of langchain-python-docs-text-embedding-ada-002
        metric='dotproduct',
        spec=spec
    )

# wait for index to be initialized
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

Then connect to the index:

In [9]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

We should see that the new Pinecone index has a `total_vector_count` of `0`, as we haven't added any vectors yet.

Now we upsert the data to Pinecone:

In [10]:
index.upsert_from_dataframe(dataset.documents, batch_size=100)

sending upsert requests:   0%|          | 0/6952 [00:00<?, ?it/s]

{'upserted_count': 6952}

We've indexed everything, now we can check the number of vectors in our index like so:

In [11]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3476}},
 'total_vector_count': 3476}

## Creating a Vector Store and Querying

In [12]:
from langchain.embeddings.openai import OpenAIEmbeddings

openai_api_key = ""
model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=openai_api_key
)

/home/a6tang/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Now that we've build our index we can switch back over to LangChain. We start by initializing a vector store using the same index we just built. We do that like so:

In [13]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pc.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

/home/a6tang/.local/lib/python3.9/site-packages/langchain_community/vectorstores/pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


As in previous examples, we can use the `similarity_search` method to do a pure semantic search (without the generation component).

In [14]:
query = "What are some of the features of Langchain?"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content='LangChain is an intuitive framework created to assist in developing applications driven by a language model, such as OpenAI or Hugging Face. Missing: decentralized | Must include:decentralized. LangChain, created by Harrison Chase, is a Python library that provides out-of-the-box support to build NLP applications using LLMs. Missing: decentralized | Must include:decentralized. LangChain provides a standard interface for chains, enabling developers to create sequences of calls that go beyond a single LLM call. Chains ... Missing: decentralized platform natural. LangChain is a powerful framework that simplifies the process of building advanced language model applications. Missing: platform | Must include:platform. Are your language models ignoring previous instructions ... Duration: 32:23. Posted: Feb 21, 2023. LangChain is a framework that enables quick and easy development of applications ... Prompting is the new way of programming NLP models. Missing: decentral

Looks like we're getting good results. Let's take a look at how we can begin integrating this into a conversational agent.

## Initializing the Conversational Agent

Our conversational agent needs a Chat LLM, conversational memory, and a `RetrievalQA` chain to initialize. We create these using:

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

# chat completion llm
llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

/home/a6tang/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Using these we can generate an answer using the `run` method:

In [16]:
qa.run(query)

/home/a6tang/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Some of the features of LangChain include:\n- Providing out-of-the-box support to build NLP applications using large language models (LLMs).\n- Offering a standard interface for chains, enabling developers to create sequences of calls that go beyond a single LLM call.\n- Simplifying the process of building advanced language model applications.\n- Enabling quick and easy development of applications powered by language models.\n- Supporting prompt management, prompt optimization, and a generic interface for all LLMs.\n- Facilitating data augmented generation involving interactions with external data sources.\n- Involving agents that make decisions about actions to take based on observations.\n- Abstracting the core building blocks of LLM applications for developers.\n- Providing integrations with other tools and end-to-end chains for common applications.\n- Enabling the creation of question-answering tools based on financial data and streamable data stores.\n- Offering a framework for d

But this isn't yet ready for our conversational agent. For that we need to convert this retrieval chain into a tool. We do that like so:

In [17]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'use this tool when answering general knowledge queries to get '
            'more information about the topic'
        )
    )
]

Now we can initialize the agent like so:

In [18]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

/home/a6tang/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


With that our retrieval augmented conversational agent is ready and we can begin using it.

### Using the Conversational Agent

To make queries we simply call the `agent` directly.

In [19]:
agent(query)

/home/a6tang/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base",
    "action_input": "Langchain features"
}
```
Observation: LangChain features include:
1. Support for building NLP applications using large language models (LLMs).
2. Standard interface for chains, enabling developers to create sequences of calls that go beyond a single LLM call.
3. Data Augmented Generation capabilities for interacting with external data sources.
4. Agents functionality for LLM decision-making and action-taking.
5. Integration with other tools for end-to-end applications.
6. Prompt management and optimization for LLMs.
7. Simplified embedding creation and storage using Pinecone and Chroma.
8. Ability to connect to any model and ingest any custom database.
9. Use cases for various applications, including question-answering tools and search engines.
10. Documentation covering installation, examples, API, and core concepts.
Thought:```json
{
    "action": "Final Answer",
    "action_input"

{'input': 'What are some of the features of Langchain?',
 'chat_history': [],
 'output': 'LangChain features include: 1. Support for building NLP applications using large language models (LLMs). 2. Standard interface for chains, enabling developers to create sequences of calls that go beyond a single LLM call. 3. Data Augmented Generation capabilities for interacting with external data sources. 4. Agents functionality for LLM decision-making and action-taking. 5. Integration with other tools for end-to-end applications. 6. Prompt management and optimization for LLMs. 7. Simplified embedding creation and storage using Pinecone and Chroma. 8. Ability to connect to any model and ingest any custom database. 9. Use cases for various applications, including question-answering tools and search engines. 10. Documentation covering installation, examples, API, and core concepts.'}

Looks great, now what if we ask it a non-general knowledge question?

In [20]:
agent("what is 2 * 7?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "The result of 2 * 7 is 14."
}
```

> Finished chain.


{'input': 'what is 2 * 7?',
 'chat_history': [HumanMessage(content='What are some of the features of Langchain?'),
  AIMessage(content='LangChain features include: 1. Support for building NLP applications using large language models (LLMs). 2. Standard interface for chains, enabling developers to create sequences of calls that go beyond a single LLM call. 3. Data Augmented Generation capabilities for interacting with external data sources. 4. Agents functionality for LLM decision-making and action-taking. 5. Integration with other tools for end-to-end applications. 6. Prompt management and optimization for LLMs. 7. Simplified embedding creation and storage using Pinecone and Chroma. 8. Ability to connect to any model and ingest any custom database. 9. Use cases for various applications, including question-answering tools and search engines. 10. Documentation covering installation, examples, API, and core concepts.')],
 'output': 'The result of 2 * 7 is 14.'}

Perfect, the agent is able to recognize that it doesn't need to refer to it's general knowledge tool for that question. Let's try some more questions.

In [21]:
agent("Can you tell me some facts about how to use Langchain?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base",
    "action_input": "Langchain usage facts"
}
```
Observation: LangChain is a framework designed to assist in developing applications driven by language models like OpenAI or Hugging Face. It provides support for building NLP applications using LLMs, offers a standard interface for chains to create sequences of calls, and simplifies the process of building advanced language model applications. LangChain also supports various modules such as prompt management, chains, data augmented generation, and agents. Common use cases for LangChain include personal assistants, question answering, chatbots, querying tabular data, interacting with APIs, extraction, summarization, and evaluation. Additionally, LangChain can be used to include AI from large language models inside data pipelines and applications.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "LangChain is a framework designed to assis

{'input': 'Can you tell me some facts about how to use Langchain?',
 'chat_history': [HumanMessage(content='What are some of the features of Langchain?'),
  AIMessage(content='LangChain features include: 1. Support for building NLP applications using large language models (LLMs). 2. Standard interface for chains, enabling developers to create sequences of calls that go beyond a single LLM call. 3. Data Augmented Generation capabilities for interacting with external data sources. 4. Agents functionality for LLM decision-making and action-taking. 5. Integration with other tools for end-to-end applications. 6. Prompt management and optimization for LLMs. 7. Simplified embedding creation and storage using Pinecone and Chroma. 8. Ability to connect to any model and ingest any custom database. 9. Use cases for various applications, including question-answering tools and search engines. 10. Documentation covering installation, examples, API, and core concepts.'),
  HumanMessage(content='what 

Looks great! We're also able to ask questions that refer to previous interactions in the conversation and the agent is able to refer to the conversation history to as a source of information.

That's all for this example of building a retrieval augmented conversational agent with OpenAI and Pinecone (the OP stack) and LangChain.

Once finished, we delete the Pinecone index to save resources:

In [22]:
pc.delete_index(index_name)

---